#Collecting wikipedia articles on the occupations


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('//content//drive')

Here, we try to fetch the wikipedia article for each of the occupation to get relevant information for training. 
None of the articles have any mentions of gendered pronoun

In [ ]:
pakoccupations=pd.read_csv("/content/drive/MyDrive/Pattern Recognition/GenderBiasCheckingDatasets/pakoccs2015.csv")
banoccupations=pd.read_csv("/content/drive/MyDrive/Pattern Recognition/GenderBiasCheckingDatasets/banoccs2020.csv")
afgoccupations=pd.read_csv("/content/drive/MyDrive/Pattern Recognition/GenderBiasCheckingDatasets/jobsACBAR.csv")

In [ ]:
!pip3 install wikipedia-api

In [ ]:
import wikipediaapi

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [ ]:
banoccupations.columns

In [ ]:
#banoccupations.insert(6, "WikipediaArticle", "")

In [ ]:
'''ind=0
for occupation in banoccupations["Unit group description (English)"]:
  if banoccupations['WikipediaArticle'].values[ind]=="":
    p_wiki = wiki_wiki.page(occupation)
    banoccupations['WikipediaArticle'].values[ind]=p_wiki.text
  ind=ind+1'''

In [ ]:
ind=0
filled=0
for occupation in banoccupations['WikipediaArticle']:
  if occupation=="":
    ind=ind+1
  else:
    filled=filled+1
print(ind, filled)

In [ ]:
banoccupations.head(20)

In [ ]:
pakoccupations.columns

In [ ]:
#pakoccupations.insert(6, "WikipediaArticle", "")

In [ ]:
'''ind=0
for occupation in pakoccupations["Unit group description (English)"]:
  if pakoccupations['WikipediaArticle'].values[ind]=="":
    p_wiki = wiki_wiki.page(occupation)
    pakoccupations['WikipediaArticle'].values[ind]=p_wiki.text
  ind=ind+1'''

In [ ]:
ind=0
filled=0
for occupation in pakoccupations['WikipediaArticle']:
  if occupation=="":
    ind=ind+1
  else:
    filled=filled+1
print(ind, filled)

In [ ]:
'''pakoccupations.insert(6, "maleMentions", 0)
pakoccupations.insert(7, "femaleMentions", 0)'''

In [ ]:
'''ind=0
if " He " in pakoccupations["WikipediaArticle"] or " he " in pakoccupations["WikipediaArticle"] or " He's " in pakoccupations["WikipediaArticle"] or " he's " in pakoccupations["WikipediaArticle"] or " Him " in pakoccupations["WikipediaArticle"] or " him " in pakoccupations["WikipediaArticle"]:
  pakoccupations["maleMentions"].values[ind]=1
if " She " in pakoccupations["WikipediaArticle"] or " she " in pakoccupations["WikipediaArticle"] or " She's " in pakoccupations["WikipediaArticle"] or " she's " in pakoccupations["WikipediaArticle"] or " Her " in pakoccupations["WikipediaArticle"] or " her " in pakoccupations["WikipediaArticle"]:
  pakoccupations["femaleMentions"].values[ind]=1'''

In [ ]:
pakoccupations.dtypes

In [ ]:
'''males=0
females=0
for mention in pakoccupations['maleMentions']:
  males=males+mention

for mention in pakoccupations['femaleMentions']:
  females=females+mention

print(males, females)'''

In [ ]:
pakoccupations.head(20)

In [ ]:
afgoccupations.columns

In [ ]:
#afgoccupations.insert(6, "WikipediaArticle", "")

In [ ]:
'''ind=0
for occupation in afgoccupations["JobPos"]:
  p_wiki = wiki_wiki.page(occupation)
  afgoccupations['WikipediaArticle'].values[ind]=p_wiki.text
  ind=ind+1'''

In [ ]:
ind=0
filled=0
for occupation in afgoccupations['WikipediaArticle']:
  if occupation=="":
    ind=ind+1
  else:
    filled=filled+1
print(ind, filled)

#Own model (With custom dataset)

In [ ]:
!pip install "tensorflow-text>=2.11"
!pip install einops

In [ ]:
import numpy as np

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

In [ ]:
#@title
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)
      
      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

##Bengali

###Getting the context-target pair

In [ ]:
def load_data(path):
  text = open(path, "r", encoding='utf-8')
  context=[]
  target=[]
  for line in text:
    pairs = line.split('\t')
    context.append(pairs[1])
    target.append(pairs[0])
  context=np.array(context)
  target=np.array(target)
  return target, context

In [ ]:
targetRaw, contextRaw = load_data('/content/drive/MyDrive/Pattern Recognition/MultilingualCorpora/RandomNWiki/combinedWikiRandEn-Bn.txt')
print(contextRaw[-1])

In [ ]:
print(targetRaw[-1])

###Test Train split

In [ ]:
BUFFER_SIZE = len(contextRaw)
BATCH_SIZE = 64

isTrain = np.random.uniform(size=(len(targetRaw),)) < 0.8

trainRaw = (
    tf.data.Dataset
    .from_tensor_slices((contextRaw[isTrain], targetRaw[isTrain]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((contextRaw[~isTrain], targetRaw[~isTrain]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [ ]:
for exampleContextStrings, exampleTargetStrings in trainRaw.take(1):
  print(exampleContextStrings[:5])
  print()
  print(exampleTargetStrings[:5])
  break

###Text Preprocessing

###Standardization

In [ ]:
exampleText = tf.constant('রুমে চলে যান।')

print(exampleText.numpy())
print(tf_text.normalize_utf8(exampleText, 'NFKC').numpy())

Unicode Normalization (For all languages)

In [ ]:
def lowerSplitPunct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKC')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ \u0980-\u09FFa-z۔؟،«»।ا.?!,]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[۔؟،«»ا।.?!,]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [ ]:
print(exampleText.numpy().decode())
print(lowerSplitPunct(exampleText).numpy().decode())

Text Vectorization

In [ ]:
maxVocabSize = 5000

contextTextProcessor = tf.keras.layers.TextVectorization(
    standardize=lowerSplitPunct,
    max_tokens=maxVocabSize,
    ragged=True)

Train (adapt=~  Model.fit)

In [ ]:
contextTextProcessor.adapt(trainRaw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
contextTextProcessor.get_vocabulary()[:10]

FOR ENGLISH

In [ ]:
targetTextProcessor = tf.keras.layers.TextVectorization(
    standardize=lowerSplitPunct,
    max_tokens=maxVocabSize,
    ragged=True)

targetTextProcessor.adapt(trainRaw.map(lambda context, target: target))
targetTextProcessor.get_vocabulary()[:10]

Strings batch to tokens

In [ ]:
exampleTokens = contextTextProcessor(exampleContextStrings)
exampleTokens[:3, :]

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(exampleTokens.to_tensor())
plt.title('Token IDs')

plt.subplot(1, 2, 2)
plt.pcolormesh(exampleTokens.to_tensor() != 0)
plt.title('Mask')

###Dataset processing

In [ ]:
def processText(context, target):
  context = contextTextProcessor(context).to_tensor()
  target = targetTextProcessor(target)
  targIn = target[:,:-1].to_tensor()
  targOut = target[:,1:].to_tensor()
  return (context, targIn), targOut


trainDS = trainRaw.map(processText, tf.data.AUTOTUNE)
valDS = val_raw.map(processText, tf.data.AUTOTUNE)

In [ ]:
for (exContextTok, exTarIn), exTarOut in trainDS.take(1):
  print(exContextTok[0, :10].numpy()) 
  print()
  print(exTarIn[0, :10].numpy()) 
  print(exTarOut[0, :10].numpy()) 

Encoder-Decoder

In [ ]:
UNITS = 400

Encoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, text_processor, units):
    super(Encoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.units = units
    
    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size, units,
                                               mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='mul',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, x):
    shape_checker = ShapeChecker()
    shape_checker(x, 'batch s')

    # 2. The embedding layer looks up the embedding vector for each token.
    x = self.embedding(x)
    shape_checker(x, 'batch s units')

    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    shape_checker(x, 'batch s units')

    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    context = self.text_processor(texts).to_tensor()
    context = self(context)
    return context

Encoder check

In [ ]:
# Encode the input sequence.
encoder = Encoder(contextTextProcessor, UNITS)
exContext = encoder(exContextTok)

print(f'Context tokens, shape (batch, s): {exContextTok.shape}')
print(f'Encoder output, shape (batch, s, units): {exContext.shape}')

Attention Layer

In [ ]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=2, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    shape_checker = ShapeChecker()
 
    shape_checker(x, 'batch t units')
    shape_checker(context, 'batch s units')

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)
    
    shape_checker(x, 'batch t units')
    shape_checker(attn_scores, 'batch heads t s')
    
    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
attentionLayer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(targetTextProcessor.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)
exTarEmbed = embed(exTarIn)

result = attentionLayer(exTarEmbed, exContext)

print(f'Context sequence, shape (batch, s, units): {exContext.shape}')
print(f'Target sequence, shape (batch, t, units): {exTarEmbed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attentionLayer.last_attention_weights.shape}')

In [ ]:
attentionLayer.last_attention_weights[0].numpy().sum(axis=-1)

In [ ]:
attentionWeights = attentionLayer.last_attention_weights
mask=(exContextTok != 0).numpy()

plt.subplot(1, 2, 1)
plt.pcolormesh(mask*attentionWeights[:, 0, :])
plt.title('Attention weights')

plt.subplot(1, 2, 2)
plt.pcolormesh(mask)
plt.title('Mask');


Decoder

In [ ]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

In [ ]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  shape_checker = ShapeChecker()
  shape_checker(x, 'batch t')
  shape_checker(context, 'batch s units')

  # 1. Lookup the embeddings
  x = self.embedding(x)
  shape_checker(x, 'batch t units')

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  shape_checker(x, 'batch t units')

  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights
  shape_checker(x, 'batch t units')
  shape_checker(self.last_attention_weights, 'batch t s')

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)
  shape_checker(logits, 'batch t target_vocab_size')

  if return_state:
    return logits, state
  else:
    return logits

In [ ]:
decoder = Decoder(targetTextProcessor, UNITS)

Given the context and target tokens, for each target token it predicts the next target token.

Decoder check

In [ ]:
logits = decoder(exContext, exTarIn)

print(f'encoder output shape: (batch, s, units) {exContext.shape}')
print(f'input target tokens shape: (batch, t) {exTarIn.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

INFERENCE

In [ ]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [ ]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [ ]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 
  
  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)
  
  return next_token, done, state

Generation Loop

In [ ]:
# Setup the loop variables.
nextToken, done, state = decoder.get_initial_state(exContext)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      exContext, nextToken, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

Actual Model

In [ ]:
class Translator(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units,
               context_text_processor,
               target_text_processor):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(context_text_processor, units)
    decoder = Decoder(target_text_processor, units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self, inputs):
    context, x = inputs
    context = self.encoder(context)
    logits = self.decoder(context, x)

    #TODO(b/250038731): remove this
    try:
      # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
      del logits._keras_mask
    except AttributeError:
      pass

    return logits

In [ ]:
model = Translator(UNITS, contextTextProcessor, targetTextProcessor)

logits = model((exContextTok, exTarIn))

print(f'Context tokens, shape: (batch, s, units) {exContextTok.shape}')
print(f'Target tokens, shape: (batch, t) {exTarIn.shape}')
print(f'logits, shape: (batch, t, targetVocabularySize) {logits.shape}')

Training

loss function

In [ ]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

accuracy function

In [ ]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)
    
    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)
    
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

Model Config with 
Adam optimizer and metrics

In [ ]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [ ]:
vocabSize = 1.0 * targetTextProcessor.vocabulary_size()

{"expected_loss": tf.math.log(vocabSize).numpy(),
 "expected_acc": 1/vocabSize}

In [ ]:
model.evaluate(valDS, steps=20, return_dict=True)

TRAINING THE MODEL (FINALLY)

In [ ]:
history = model.fit(
    trainDS.repeat(), 
    epochs=40,
    steps_per_epoch = 130,
    validation_data=valDS,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
plt.plot(history.history['masked_acc'], label='accuracy')
plt.plot(history.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

Translate

In [ ]:
#@title
@Translator.add_method
def translate(self,
              texts, *,
              max_length=50,
              temperature=0.0):
  # Process the input texts
  context = self.encoder.convert_input(texts)
  batch_size = tf.shape(texts)[0]

  # Setup the loop inputs
  tokens = []
  attention_weights = []
  next_token, done, state = self.decoder.get_initial_state(context)

  for _ in range(max_length):
    # Generate the next token
    next_token, done, state = self.decoder.get_next_token(
        context, next_token, done,  state, temperature)
        
    # Collect the generated tokens
    tokens.append(next_token)
    attention_weights.append(self.decoder.last_attention_weights)
    
    if tf.executing_eagerly() and tf.reduce_all(done):
      break

  # Stack the lists of tokens and attention weights.
  tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
  self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

  result = self.decoder.tokens_to_text(tokens)
  return result

In [ ]:
result = model.translate(['তিনি একজন আইনজীবী'])
result[0].numpy().decode()

In [ ]:
result = model.translate(['সে একজন ডাক্তার হবে'])
result[0].numpy().decode()

In [ ]:

result = model.translate(['তিনি একজন পাইলট'])
result[0].numpy().decode()

In [ ]:

result = model.translate(['তিনি একজন প্রশিক্ষক'])
result[0].numpy().decode()

In [ ]:

result = model.translate(['গতকাল সেই ব্যক্তিকে দেখেছি'])
result[0].numpy().decode()

In [ ]:

result = model.translate(['গতকাল সেই লোকটিকে দেখেছি। সে একজন ডাক্তার.'])
result[0].numpy().decode()

In [ ]:
#@title
@Translator.add_method
def plot_attention(self, text, **kwargs):
  assert isinstance(text, str)
  output = self.translate([text], **kwargs)
  output = output[0].numpy().decode()

  attention = self.last_attention_weights[0]

  context = lowerSplitPunct(text)
  context = context.numpy().decode().split()

  output = lowerSplitPunct(output)
  output = output.numpy().decode().split()[1:]

  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)

  ax.matshow(attention, cmap='viridis', vmin=0.0)

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + context, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + output, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  ax.set_xlabel('Input text')
  ax.set_ylabel('Output text')

In [ ]:
model.plot_attention('তিনি একজন আইনজীবী') # Are you still home

In [ ]:
%%time
# This is my life.
model.plot_attention('এটাই আমার জীবন.')

###Bias Performance check

In [ ]:
banoccupations=banoccupations.drop('WikipediaArticle', axis=1)

In [ ]:
banoccupations.head()

In [ ]:
banoccupations.columns

In [ ]:
banoccupations.insert(10, "ModelTranslation(SingleSentence)", "")

In [ ]:
ind=0
for sent in banoccupations['Sent(Bn)']:
  result = model.translate([sent])
  result=result[0].numpy().decode()
  banoccupations['ModelTranslation(SingleSentence)'].values[ind]=result
  ind=ind+1

In [ ]:
banoccupations.insert(10, "ModelTranslation(MultipleSentences)", "")

In [ ]:
ind=0
for sent in banoccupations['SentWithCont(Bn)']:
  result = model.translate([sent])
  result=result[0].numpy().decode()
  banoccupations['ModelTranslation(MultipleSentences)'].values[ind]=result
  ind=ind+1

In [ ]:
banoccupations.insert(10, "ModelPronoun(SingleSentence)", "")

In [ ]:
ind=0
for val in banoccupations['ModelTranslation(SingleSentence)']:
  #print(ind, val)
  if (' She ' in val) or ((" She's " in val)) or (' she ' in val) or ((" she's " in val)):
    banoccupations['ModelPronoun(SingleSentence)'].values[ind]='F'
  elif (' He ' in val) or ((" He's " in val)) or (' he ' in val) or ((" he's " in val)):
    banoccupations['ModelPronoun(SingleSentence)'].values[ind]='M'
  else:
    banoccupations['ModelPronoun(SingleSentence)'].values[ind]='O'
  ind=ind+1

In [ ]:
banoccupations.insert(10, "ModelPronoun(MultipleSentences)", "")

In [ ]:
ind=0
for val in banoccupations['ModelTranslation(SingleSentence)']:
  #print(ind, val)
  if ('She ' in val) or (("She's " in val)) or ('she ' in val) or (("she's " in val)):
    banoccupations['ModelPronoun(SingleSentence)'].values[ind]='F'
  elif ('He ' in val) or (("He's " in val)) or ('he ' in val) or (("he's " in val)):
    banoccupations['ModelPronoun(SingleSentence)'].values[ind]='M'
  else:
    banoccupations['ModelPronoun(SingleSentence)'].values[ind]='O'
  ind=ind+1

In [ ]:
def autopct(pct): # only show the label when it's > 10%
    return ('%.2f' % pct) if pct > 10 else ''
'''labels=('Female', 'Male', 'Others')
afgoccupations_ax=afgoccupations['Pronoun(GTranslateAlt)'].value_counts().plot(kind='pie', figsize=(13,8), autopct=autopct, title='Male, Female and Other pronouns count in the Alternative single sentence translation by Google Translate', labels=None)
afgoccupations_ax.axes.get_yaxis().set_visible(False)
plt.legend(loc=5, labels=labels)'''

In [ ]:
banoccupations_ax=banoccupations['ModelPronoun(SingleSentence)'].value_counts().plot(kind='pie', autopct=autopct, title='Male, Female and Other pronouns count in the translation without context(Bangla)')
plt.show()

In [ ]:
banoccupations['ModelPronoun(SingleSentence)'].value_counts()

In [ ]:
banoccupations_ax=banoccupations['ModelPronoun(MultipleSentences)'].value_counts().plot(kind='pie', autopct=autopct, title='Male, Female and Other pronouns count in the translation with context(Bangla)')
plt.show()

In [ ]:
banoccupations['ModelPronoun(MultipleSentences)'].value_counts()

##Urdu

###Getting the context-target pair

In [ ]:
targetRaw, contextRaw = load_data('/content/drive/MyDrive/Pattern Recognition/MultilingualCorpora/RandomNWiki/combinedWikiRandEn-Ur.txt')
print(contextRaw[25])

In [ ]:
print(targetRaw[25])

###Test Train split

In [ ]:
BUFFER_SIZE = len(contextRaw)
BATCH_SIZE = 64

isTrain = np.random.uniform(size=(len(targetRaw),)) < 0.8

trainRaw = (
    tf.data.Dataset
    .from_tensor_slices((contextRaw[isTrain], targetRaw[isTrain]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
valRaw = (
    tf.data.Dataset
    .from_tensor_slices((contextRaw[~isTrain], targetRaw[~isTrain]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [ ]:
for exampleContextStrings, exampleTargetStrings in trainRaw.take(1):
  print(exampleContextStrings[:5])
  print()
  print(exampleTargetStrings[:5])
  break

###Text Preprocessing

###Standardization

In [ ]:
exampleText = tf.constant('آدمی')

print(exampleText.numpy())
print(tf_text.normalize_utf8(exampleText, 'NFKC').numpy())

Unicode Normalization (For all languages)

In [ ]:
print(exampleText.numpy().decode())
print(lowerSplitPunct(exampleText).numpy().decode())

Text Vectorization

Train (adapt=~  Model.fit)

In [ ]:
contextTextProcessor.adapt(trainRaw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
contextTextProcessor.get_vocabulary()[:50]

FOR ENGLISH

In [ ]:
targetTextProcessor.adapt(trainRaw.map(lambda context, target: target))
targetTextProcessor.get_vocabulary()[:10]

Strings batch to tokens

In [ ]:
exampleTokens = contextTextProcessor(exampleContextStrings)
exampleTokens[:3, :]

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(exampleTokens.to_tensor())
plt.title('Token IDs')

plt.subplot(1, 2, 2)
plt.pcolormesh(exampleTokens.to_tensor() != 0)
plt.title('Mask')

###Dataset processing

Vectorization

In [ ]:
trainDS= trainRaw.map(processText, tf.data.AUTOTUNE)
valDS= valRaw.map(processText, tf.data.AUTOTUNE)

In [ ]:
for (exContextTok, exTarIn), exTarOut in trainDS.take(1):
  print(exContextTok[0, :10].numpy()) 
  print()
  print(exTarIn[0, :10].numpy()) 
  print(exTarOut[0, :10].numpy()) 

Encoder-Decoder

Encoder check

In [ ]:
# Encode the input sequence.
encoder = Encoder(contextTextProcessor, UNITS)
exContext = encoder(exContextTok)

print(f'Context tokens, shape (batch, s): {exContextTok.shape}')
print(f'Encoder output, shape (batch, s, units): {exContext.shape}')

Attention Layer

In [ ]:
attentionLayer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(targetTextProcessor.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)
exTarEmbed = embed(exTarIn)

result = attentionLayer(exTarEmbed, exContext)

print(f'Context sequence, shape (batch, s, units): {exContext.shape}')
print(f'Target sequence, shape (batch, t, units): {exTarEmbed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attentionLayer.last_attention_weights.shape}')

In [ ]:
attentionLayer.last_attention_weights[0].numpy().sum(axis=-1)

In [ ]:
attentionWeights = attentionLayer.last_attention_weights
mask=(exContextTok != 0).numpy()

plt.subplot(1, 2, 1)
plt.pcolormesh(mask*attentionWeights[:, 0, :])
plt.title('Attention weights')

plt.subplot(1, 2, 2)
plt.pcolormesh(mask)
plt.title('Mask');


Given the context and target tokens, for each target token it predicts the next target token.

Decoder check

In [ ]:
logits = decoder(exContext, exTarIn)

print(f'encoder output shape: (batch, s, units) {exContext.shape}')
print(f'input target tokens shape: (batch, t) {exTarIn.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

Generation Loop

In [ ]:
# Setup the loop variables.
nextToken, done, state = decoder.get_initial_state(exContext)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      exContext, nextToken, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

Actual Model

In [ ]:
modelUr = Translator(UNITS, contextTextProcessor, targetTextProcessor)

logits = modelUr((exContextTok, exTarIn))

print(f'Context tokens, shape: (batch, s, units) {exContextTok.shape}')
print(f'Target tokens, shape: (batch, t) {exTarIn.shape}')
print(f'logits, shape: (batch, t, targetVocabularySize) {logits.shape}')

Training

Model Config with 
Adam optimizer and metrics

In [ ]:
modelUr.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [ ]:
vocabSize = 1.0 * targetTextProcessor.vocabulary_size()

{"expected_loss": tf.math.log(vocabSize).numpy(),
 "expected_acc": 1/vocabSize}

In [ ]:
modelUr.evaluate(valDS, steps=20, return_dict=True)

TRAINING THE MODEL (FINALLY)

In [ ]:
historyUr = modelUr.fit(
    trainDS.repeat(), 
    epochs=40,
    steps_per_epoch = 130,
    validation_data=valDS,
    validation_steps = 30,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
plt.plot(historyUr.history['loss'], label='loss')
plt.plot(historyUr.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
plt.plot(historyUr.history['masked_acc'], label='accuracy')
plt.plot(historyUr.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

Translate

In [ ]:
result = modelUr.translate(['وہ ایک وکیل ہے۔'])
result[0].numpy().decode()

In [ ]:
result = modelUr.translate(['وہ ایک پائلٹ ہے۔'])
result[0].numpy().decode()

In [ ]:

result = modelUr.translate(['وہ پائلٹ ہو گی۔'])
result[0].numpy().decode()

In [ ]:
result = modelUr.translate(['وہ کوچ ہو گی۔'])
result[0].numpy().decode()

In [ ]:
result = modelUr.translate(['میں نے کل اس آدمی کو دیکھا۔'])
result[0].numpy().decode()

In [ ]:
modelUr.plot_attention('وہ ایک وکیل ہے۔')

###Bias Performance check

In [ ]:
pakoccupations=pakoccupations.drop('WikipediaArticle', axis=1)

In [ ]:
pakoccupations.head(20)

In [ ]:
pakoccupations.columns

In [ ]:
pakoccupations.insert(10, "ModelTranslation(SingleSentence)", "")

In [ ]:
pakoccupations.columns

In [ ]:
ind=0
for sent in pakoccupations['Urdu(Sent)']:
  result = modelUr.translate([sent])
  result=result[0].numpy().decode()
  pakoccupations['ModelTranslation(SingleSentence)'].values[ind]=result
  ind=ind+1

In [ ]:
pakoccupations.insert(10, "ModelTranslation(MultipleSentences)", "")

In [ ]:
ind=0
for sent in pakoccupations['Urdu(SentWithCont)']:
  result = modelUr.translate([sent])
  result=result[0].numpy().decode()
  pakoccupations['ModelTranslation(MultipleSentences)'].values[ind]=result
  ind=ind+1

In [ ]:
pakoccupations.insert(10, "ModelPronoun(SingleSentence)", "")

In [ ]:
ind=0
for val in pakoccupations['ModelTranslation(SingleSentence)']:
  #print(ind, val)
  if (' She ' in val) or ((" She's " in val)) or (' she ' in val) or ((" she's " in val)):
    pakoccupations['ModelPronoun(SingleSentence)'].values[ind]='F'
  elif (' He ' in val) or ((" He's " in val)) or (' he ' in val) or ((" he's " in val)):
    pakoccupations['ModelPronoun(SingleSentence)'].values[ind]='M'
  else:
    pakoccupations['ModelPronoun(SingleSentence)'].values[ind]='O'
  ind=ind+1

In [ ]:
pakoccupations.insert(10, "ModelPronoun(MultipleSentences)", "")

In [ ]:
ind=0
for val in pakoccupations['ModelTranslation(SingleSentence)']:
  #print(ind, val)
  if ('She ' in val) or (("She's " in val)) or ('she ' in val) or (("she's " in val)):
    pakoccupations['ModelPronoun(SingleSentence)'].values[ind]='F'
  elif ('He ' in val) or (("He's " in val)) or ('he ' in val) or (("he's " in val)):
    pakoccupations['ModelPronoun(SingleSentence)'].values[ind]='M'
  else:
    pakoccupations['ModelPronoun(SingleSentence)'].values[ind]='O'
  ind=ind+1

In [ ]:
pakoccupations_ax=pakoccupations['ModelPronoun(SingleSentence)'].value_counts().plot(kind='pie', autopct=autopct, title='Male, Female and Other pronouns count in the translation without context(Urdu)')
plt.show()

In [ ]:
pakoccupations['ModelPronoun(SingleSentence)'].value_counts()

In [ ]:
pakoccupations_ax=pakoccupations['ModelPronoun(MultipleSentences)'].value_counts().plot(kind='pie', autopct=autopct, title='Male, Female and Other pronouns count in the translation with context(Bangla)')
plt.show()

In [ ]:
pakoccupations['ModelPronoun(MultipleSentences)'].value_counts()

##Dari/Farsi

###Getting the context-target pair

In [ ]:
targetRaw, contextRaw = load_data('/content/drive/MyDrive/Pattern Recognition/MultilingualCorpora/RandomNWiki/combinedWikiRandEn-Fa.txt')
print(contextRaw[-1])

In [ ]:
print(targetRaw[-1])

###Test Train split

In [ ]:
BUFFER_SIZE = len(contextRaw)
BATCH_SIZE = 64

isTrain = np.random.uniform(size=(len(targetRaw),)) < 0.8

trainRaw = (
    tf.data.Dataset
    .from_tensor_slices((contextRaw[isTrainDa], targetRaw[isTrainDa]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
valraw = (
    tf.data.Dataset
    .from_tensor_slices((contextRaw[~isTrainDa], targetRaw[~isTrainDa]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [ ]:
for exampleContextStrings, exampleTargetStrings in trainRawDa.take(1):
  print(exampleContextStrings[:5])
  print()
  print(exampleTargetStrings[:5])
  break

###Text Preprocessing

###Standardization

In [ ]:
exampleText = tf.constant('وجود دارد.')

print(exampleText.numpy())
print(tf_text.normalize_utf8(exampleText, 'NFKC').numpy())

Unicode Normalization (For all languages)

In [ ]:
print(exampleText.numpy().decode())
print(lowerSplitPunct(exampleText).numpy().decode())

Text Vectorization

Train (adapt=~  Model.fit)

In [ ]:
contextTextProcessor.adapt(trainRawDa.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
contextTextProcessor.get_vocabulary()[:10]

FOR ENGLISH

In [ ]:
targetTextProcessor.adapt(trainRawDa.map(lambda context, target: target))
targetTextProcessor.get_vocabulary()[:10]

Strings batch to tokens

In [ ]:
exampleTokens = contextTextProcessor(exampleContextStrings)
exampleTokens[:3, :]

In [ ]:
plt.subplot(1, 2, 1)
plt.pcolormesh(exampleTokens.to_tensor())
plt.title('Token IDs')

plt.subplot(1, 2, 2)
plt.pcolormesh(exampleTokens.to_tensor() != 0)
plt.title('Mask')

###Dataset processing

Vectorization

In [ ]:
trainDS= trainRaw.map(processText, tf.data.AUTOTUNE)
valDS= valraw.map(processText, tf.data.AUTOTUNE)

In [ ]:
for (exContextTok, exTarIn), exTarOut in trainDS.take(1):
  print(exContextTok[0, :10].numpy()) 
  print()
  print(exTarIn[0, :10].numpy()) 
  print(exTarOut[0, :10].numpy()) 

Encoder-Decoder

Encoder check

In [ ]:
# Encode the input sequence.
encoder = Encoder(contextTextProcessor, UNITS)
exContext = encoder(exContextTok)

print(f'Context tokens, shape (batch, s): {exContextTok.shape}')
print(f'Encoder output, shape (batch, s, units): {exContext.shape}')

Attention Layer

In [ ]:
'''attentionLayer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(targetTextProcessor.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)'''
exTarEmbed = embed(exTarIn)

result = attentionLayer(exTarEmbed, exContext)

print(f'Context sequence, shape (batch, s, units): {exContext.shape}')
print(f'Target sequence, shape (batch, t, units): {exTarEmbed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attentionLayer.last_attention_weights.shape}')

In [ ]:
attentionLayer.last_attention_weights[0].numpy().sum(axis=-1)

In [ ]:
attentionWeights = attentionLayer.last_attention_weights
mask=(exContextTok != 0).numpy()

plt.subplot(1, 2, 1)
plt.pcolormesh(mask*attentionWeights[:, 0, :])
plt.title('Attention weights')

plt.subplot(1, 2, 2)
plt.pcolormesh(mask)
plt.title('Mask');


Given the context and target tokens, for each target token it predicts the next target token.

Decoder check

In [ ]:
logits = decoder(exContext, exTarIn)

print(f'encoder output shape: (batch, s, units) {exContext.shape}')
print(f'input target tokens shape: (batch, t) {exTarIn.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

Generation Loop

In [ ]:
# Setup the loop variables.
nextToken, done, state = decoder.get_initial_state(exContext)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      exContext, nextToken, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

Actual Model

In [ ]:
modelDa = Translator(UNITS, contextTextProcessor, targetTextProcessor)

logits = modelDa((exContextTok, exTarIn))

print(f'Context tokens, shape: (batch, s, units) {exContextTok.shape}')
print(f'Target tokens, shape: (batch, t) {exTarIn.shape}')
print(f'logits, shape: (batch, t, targetVocabularySize) {logits.shape}')

Training

Model Config with 
Adam optimizer and metrics

In [ ]:
modelDa.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [ ]:
vocabSize = 1.0 * targetTextProcessor.vocabulary_size()

{"expected_loss": tf.math.log(vocabSize).numpy(),
 "expected_acc": 1/vocabSize}

In [ ]:
modelDa.evaluate(valDS, steps=20, return_dict=True)

TRAINING THE MODEL (FINALLY)

In [ ]:
historyDa = modelDa.fit(
    trainDS.repeat(), 
    epochs=40,
    steps_per_epoch = 130,
    validation_data=valDS,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
plt.plot(historyDa.historyDa['loss'], label='loss')
plt.plot(historyDa.historyDa['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

In [ ]:
plt.plot(historyDa.historyDa['masked_acc'], label='accuracy')
plt.plot(historyDa.historyDa['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('CE/token')
plt.legend()

Translate

In [ ]:
result = modelDa.translate(['او یک وکیل است'])
result[0].numpy().decode()

In [ ]:
result = modelDa.translate(['او یک خلبان است'])
result[0].numpy().decode()

In [ ]:

result = modelDa.translate(['او مربی خواهد شد.'])
result[0].numpy().decode()

In [ ]:
result = modelDa.translate(['دیروز آن مرد را دیدم.'])
result[0].numpy().decode()

In [ ]:
modelUr.plot_attention('او یک خلبان است.')

###Bias Performance check

In [ ]:
afgoccupations=afgoccupations.drop('WikipediaArticle', axis=1)

In [ ]:
afgoccupations.head()

In [ ]:
afgoccupations.columns

In [ ]:
afgoccupations.insert(10, "ModelTranslation(SingleSentence)", "")

In [ ]:
ind=0
for sent in afgoccupations['Sent(Bn)']:
  result = modelDa.translate([sent])
  result=result[0].numpy().decode()
  afgoccupations['ModelTranslation(SingleSentence)'].values[ind]=result
  ind=ind+1

In [ ]:
pakoccupations.insert(10, "ModelTranslation(MultipleSentences)", "")

In [ ]:
afgoccupations.columns

In [ ]:
ind=0
for sent in afgoccupations['SentWithCont(Bn)']:
  result = modelDa.translate([sent])
  result=result[0].numpy().decode()
  pakoccupations['ModelTranslation(MultipleSentences)'].values[ind]=result
  ind=ind+1

In [ ]:
afgoccupations.insert(10, "ModelPronoun(SingleSentence)", "")

In [ ]:
ind=0
for val in afgoccupations['ModelTranslation(SingleSentence)']:
  #print(ind, val)
  if (' She ' in val) or ((" She's " in val)) or (' she ' in val) or ((" she's " in val)):
    afgoccupations['ModelPronoun(SingleSentence)'].values[ind]='F'
  elif (' He ' in val) or ((" He's " in val)) or (' he ' in val) or ((" he's " in val)):
    afgoccupations['ModelPronoun(SingleSentence)'].values[ind]='M'
  else:
    afgoccupations['ModelPronoun(SingleSentence)'].values[ind]='O'
  ind=ind+1

In [ ]:
afgoccupations.insert(10, "ModelPronoun(MultipleSentences)", "")

In [ ]:
ind=0
for val in afgoccupations['ModelTranslation(SingleSentence)']:
  #print(ind, val)
  if ('She ' in val) or (("She's " in val)) or ('she ' in val) or (("she's " in val)):
    afgoccupations['ModelPronoun(SingleSentence)'].values[ind]='F'
  elif ('He ' in val) or (("He's " in val)) or ('he ' in val) or (("he's " in val)):
    afgoccupations['ModelPronoun(SingleSentence)'].values[ind]='M'
  else:
    afgoccupations['ModelPronoun(SingleSentence)'].values[ind]='O'
  ind=ind+1

In [ ]:
afgoccupations_ax=afgoccupations['ModelPronoun(SingleSentence)'].value_counts().plot(kind='pie', autopct=autopct, title='Male, Female and Other pronouns count in the translation without context(Bangla)')
plt.show()

In [ ]:
afgoccupations['ModelPronoun(SingleSentence)'].value_counts()

In [ ]:
afgoccupations_ax=afgoccupations['ModelPronoun(MultipleSentences)'].value_counts().plot(kind='pie', autopct=autopct, title='Male, Female and Other pronouns count in the translation with context(Bangla)')
plt.show()

In [ ]:
afgoccupations['ModelPronoun(MultipleSentences)'].value_counts()

#BLEU Metrics

### 1. Libraries
*Install and import necessary libraries*


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

### 2. Dataset
*Array of words: candidate and reference sentences split into words*

In [ ]:
hyp = str('she read the book because she was interested in world history').split()
ref_a = str('she read the book because she was interested in world history').split()
ref_b = str('she was interested in world history because she read the book').split()